### Solutions

#### Question 1

Write a custom class that will be used to model a single bank account.

Your class should implement functionality to:
- allow initialization with values for `first_name`, `last_name`, `account_number`, `balance`, `is_overdraft_allowed`
- keep track of a "ledger" that keeps a record all transactions (just use a list to keep track of this)
    - at a minimum it should keep track of the transaction date (the current UTC datetime) and the amount (positive, or negative to indicate deposits/withdrawals) - later you could add tracking the running balance as well.
- provide read-only properties for `first_name`, `last_name`, `account_number` and `balance`
- provide a property to access the ledger in such a way that a user of this class cannot mutate the ledger directly
- provide a read-write property for `is_overdraft_allowed` that indicates whether overdrafts are allowed on the account.
- provide methods to debit (`def withdraw`) and credit (`def deposit`) transactions that:
    - verify withdrawals against available balance and `is_overdraft_allowed` flag
        - if withdrawal is larger than available balance and overdrafts are not allowed, this should raise a custom `OverdraftNotAllowed` exception.
        - if transaction value is not positive, this should raise a `ValueError` exception (we have separate methods for deposits and withdrawals, and we expect the value to be positive in both cases - one will add to the balance, one will subtract from the balance).
    - add an entry to the ledger with a current UTC timestamp (positive or negative to indicate credit/debit)
    - keeps the available balance updated
- implements a good string representation for the instance (maybe something like `first_name last_name (account_number): balance`

Feel free to expand on the minimum definition I have given here and enhance your custom class.

In [1]:
from __future__ import annotations

from datetime import datetime, timezone
from decimal import Decimal, ROUND_HALF_UP, getcontext
from typing import List, Tuple, NamedTuple

getcontext().prec = 28

class OverdraftNotAllowed(Exception):
    """Raised when a withdrawal would overdraw a non-overdraft account."""

class Transaction(NamedTuple):
    timestamp: datetime
    amount: Decimal
    balance_after: Decimal

class BankAccount:
    __slots__ = (
        '_first_name', '_last_name', '_account_number',
        '_balance', '_is_overdraft_allowed', '_ledger'
    )

    def __init__(self, first_name: str, last_name: str,
                 account_number: str, balance: float | Decimal = 0,
                 is_overdraft_allowed: bool = False) -> None:

        self._first_name = str(first_name).strip()
        self._last_name = str(last_name).strip()
        self._account_number = str(account_number).strip()

        starting_balance = self._to_decimal(balance)
        if starting_balance < Decimal('0'):
            raise ValueError('Initial balance cannot be negative.')

        self._balance = Decimal('0')
        self._is_overdraft_allowed = bool(is_overdraft_allowed)
        self._ledger = []

        if starting_balance != Decimal('0'):
            self._add_transaction(starting_balance)

    @staticmethod
    def _to_decimal(value):
        return Decimal(str(value)).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

    def _add_transaction(self, amount: Decimal) -> None:
        timestamp = datetime.now(timezone.utc)
        self._balance += amount
        self._ledger.append(Transaction(timestamp, amount, self._balance))

    @property
    def first_name(self): return self._first_name

    @property
    def last_name(self): return self._last_name

    @property
    def account_number(self): return self._account_number

    @property
    def balance(self): return self._balance

    @property
    def is_overdraft_allowed(self): return self._is_overdraft_allowed

    @is_overdraft_allowed.setter
    def is_overdraft_allowed(self, value: bool):
        self._is_overdraft_allowed = bool(value)

    @property
    def ledger(self): return tuple(self._ledger)

    def deposit(self, amount):
        dec_amount = self._to_decimal(amount)
        if dec_amount <= Decimal('0'):
            raise ValueError('Deposit amount must be positive.')
        self._add_transaction(dec_amount)

    def withdraw(self, amount):
        dec_amount = self._to_decimal(amount)
        if dec_amount <= Decimal('0'):
            raise ValueError('Withdrawal amount must be positive.')

        new_balance = self._balance - dec_amount
        if new_balance < Decimal('0') and not self._is_overdraft_allowed:
            raise OverdraftNotAllowed('Insufficient funds: overdraft not allowed.')

        self._add_transaction(-dec_amount)

    def __str__(self):
        return f"{self.first_name} {self.last_name} ({self.account_number}): {self.balance:.2f}"

    def __repr__(self):
        return (
            f"BankAccount(first_name={self.first_name!r}, last_name={self.last_name!r}, "
            f"account_number={self.account_number!r}, balance={self.balance!r}, "
            f"is_overdraft_allowed={self.is_overdraft_allowed!r})"
        )

    def __eq__(self, other):
        if not isinstance(other, BankAccount): return NotImplemented
        return self.account_number == other.account_number

    def __hash__(self): return hash(self.account_number)


#### Question 2

Expand on your class above to implement equality (`==`) comparisons between instances of your class.

Two accounts should be considered equal if the account numbers are the same.

In [2]:
acc1 = BankAccount('Alice', 'Smith', 'ACC-001', balance=100)
acc2 = BankAccount('Bob', 'Jones', 'ACC-001', balance=500)
acc3 = BankAccount('Charlie', 'Brown', 'ACC-999', balance=100)

print('acc1 == acc2:', acc1 == acc2)
print('acc1 == acc3:', acc1 == acc3)

accounts_set = {acc1, acc2, acc3}
print('Unique accounts:', len(accounts_set))

acc1 == acc2: True
acc1 == acc3: False
Unique accounts: 2
